In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv
/kaggle/input/twitter-sentiment-analysis-hatred-speech/test.csv


In [2]:
train=pd.read_csv('/kaggle/input/twitter-sentiment-analysis-hatred-speech/train.csv')
test=pd.read_csv('/kaggle/input/twitter-sentiment-analysis-hatred-speech/test.csv')

In [3]:
train.head()
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


# **Removing hashtags**

In [4]:
def punc(df):
    df['tweet'] = df['tweet'].str.replace('[#]','')
    print(df)

In [5]:
punc(train)
punc(test)

          id  label                                              tweet
0          1      0   @user when a father is dysfunctional and is s...
1          2      0  @user @user thanks for lyft credit i can't use...
2          3      0                                bihday your majesty
3          4      0  model   i love u take with u all the time in u...
4          5      0              factsguide: society now    motivation
...      ...    ...                                                ...
31957  31958      0  ate @user isz that youuu?ðððððð...
31958  31959      0    to see nina turner on the airwaves trying to...
31959  31960      0  listening to sad songs on a monday morning otw...
31960  31961      1  @user sikh temple vandalised in in calgary, ws...
31961  31962      0                   thank you @user for you follow  

[31962 rows x 3 columns]
          id                                              tweet
0      31963  studiolife aislife requires passion dedicati

# Tokenizing

In [6]:
import nltk
from nltk.tokenize import TweetTokenizer
def tokenizer(df):
    tknzr = TweetTokenizer(strip_handles=True)
    df['tweet']= df['tweet'].apply(lambda x: tknzr.tokenize(x))
    print(df)
    

In [7]:
tokenizer(test)
tokenizer(train)

          id                                              tweet
0      31963  [studiolife, aislife, requires, passion, dedic...
1      31964  [white, supremacists, want, everyone, to, see,...
2      31965  [safe, ways, to, heal, your, acne, !, !, altwa...
3      31966  [is, the, hp, and, the, cursed, child, book, u...
4      31967  [3rd, bihday, to, my, amazing, ,, hilarious, n...
...      ...                                                ...
17192  49155  [thought, factory, :, left-right, polarisation...
17193  49156  [feeling, like, a, mermaid, ð, , , , hairfl...
17194  49157  [hillary, campaigned, today, in, ohio, (, (, o...
17195  49158  [happy, ,, at, work, conference, :, right, min...
17196  49159  [my, song, ", so, glad, ", free, download, !, ...

[17197 rows x 2 columns]
          id  label                                              tweet
0          1      0  [when, a, father, is, dysfunctional, and, is, ...
1          2      0  [thanks, for, lyft, credit, i, can't, use, 

# Removing Stopwords

In [8]:
import nltk
from nltk.corpus import stopwords
stop=stopwords.words("english")
def stop_words(df):
    df['tweet']=df['tweet'].apply(lambda x: [i.lower() for i in x if i not in stop])
    print(df)

In [9]:
stop_words(train)
stop_words(test)

          id  label                                              tweet
0          1      0  [father, dysfunctional, selfish, drags, kids, ...
1          2      0  [thanks, lyft, credit, can't, use, cause, offe...
2          3      0                                  [bihday, majesty]
3          4      0  [model, love, u, take, u, time, urð, , , ±, ...
4          5      0               [factsguide, :, society, motivation]
...      ...    ...                                                ...
31957  31958      0  [ate, isz, youuu, ?, ð, , , , ð, , , , ð...
31958  31959      0  [see, nina, turner, airwaves, trying, wrap, ma...
31959  31960      0  [listening, sad, songs, monday, morning, otw, ...
31960  31961      1  [sikh, temple, vandalised, calgary, ,, wso, co...
31961  31962      0                                    [thank, follow]

[31962 rows x 3 columns]
          id                                              tweet
0      31963  [studiolife, aislife, requires, passion, ded

 Removing punctuations and words of length less than 3

In [10]:
import re
def clean(df):
    df['tweet']=df['tweet'].apply(lambda x: [i for i in x if not re.match('[^\w\s]',i) and len(i)>3])
    print(df)

In [11]:
clean(train)
clean(test)

          id  label                                              tweet
0          1      0  [father, dysfunctional, selfish, drags, kids, ...
1          2      0  [thanks, lyft, credit, can't, cause, offer, wh...
2          3      0                                  [bihday, majesty]
3          4      0                          [model, love, take, time]
4          5      0                  [factsguide, society, motivation]
...      ...    ...                                                ...
31957  31958      0                                            [youuu]
31958  31959      0  [nina, turner, airwaves, trying, wrap, mantle,...
31959  31960      0          [listening, songs, monday, morning, work]
31960  31961      1      [sikh, temple, vandalised, calgary, condemns]
31961  31962      0                                    [thank, follow]

[31962 rows x 3 columns]
          id                                              tweet
0      31963  [studiolife, aislife, requires, passion, ded

# Stemming

In [12]:
from nltk.stem import PorterStemmer
from textblob import Word
st = PorterStemmer()
def stemnlemm(df):
    df['tweet']=df['tweet'].apply(lambda x: [Word(st.stem(i)).lemmatize() for i in x])
    print(df)

In [13]:
stemnlemm(train)
stemnlemm(test)

          id  label                                              tweet
0          1      0   [father, dysfunct, selfish, drag, kid, dysfunct]
1          2      0  [thank, lyft, credit, can't, caus, offer, whee...
2          3      0                                  [bihday, majesti]
3          4      0                          [model, love, take, time]
4          5      0                        [factsguid, societi, motiv]
...      ...    ...                                                ...
31957  31958      0                                            [youuu]
31958  31959      0  [nina, turner, airwav, tri, wrap, mantl, genui...
31959  31960      0                 [listen, song, monday, morn, work]
31960  31961      1          [sikh, templ, vandalis, calgari, condemn]
31961  31962      0                                    [thank, follow]

[31962 rows x 3 columns]
          id                                              tweet
0      31963  [studiolif, aislif, requir, passion, dedic, 

# Vectorizing

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500, analyzer='word', lowercase=False) 

In [15]:
train['tweet']=train['tweet'].apply(lambda x: " ".join(x) )
test['tweet']=test['tweet'].apply(lambda x: " ".join(x) )

X_train = cv.fit_transform(train['tweet'])

In [16]:
X_train

<31962x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 138902 stored elements in Compressed Sparse Row format>

In [17]:
Y_train=pd.DataFrame(train['label'])
Y_train.head()

,label
0,0
1,0
2,0
3,0
4,0


Splitting dataset!

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.3)

In [19]:
x_test.shape

(9589, 1500)

Naive Bayes Classifier

In [20]:
from sklearn.naive_bayes import MultinomialNB
z=MultinomialNB()
z.fit(x_train,y_train)

pred=z.predict(x_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [21]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score
cm=confusion_matrix(y_test,pred)
print(cm)
score = accuracy_score( y_test, pred)
print(score)


[[8671  263]
 [ 299  356]]
0.9413911773907603


K-neighbor Classifier

In [22]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
classifier.fit(x_train, y_train)

#predicting test results
pred = classifier.predict(x_test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_test,pred)
print(cm)
score = accuracy_score( y_test, pred)
print(score)


[[8861   73]
 [ 481  174]]
0.9422254666805715


Random Forest Classifier

In [24]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier( n_estimators=100 ,criterion='entropy', random_state=0)
classifier.fit(x_train, y_train)

#predicting test results
pred = classifier.predict(x_test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_test,pred)
print(cm)
score = accuracy_score( y_test, pred)
print(score)

[[8740  194]
 [ 317  338]]
0.9467097716133069


Logistic Regression Classifier

In [26]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train, y_train)

#predicting test results
y_pred = classifier.predict(x_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [27]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_test,pred)
print(cm)
score = accuracy_score( y_test, pred)
print(score)

[[8740  194]
 [ 317  338]]
0.9467097716133069


XG Boost Classifier

In [28]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [29]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_test,y_pred)
print(cm)
score = accuracy_score( y_test, y_pred)
print(score)

[[8881   53]
 [ 412  243]]
0.9515069350297215


Support Vector Machine Classifier

In [30]:
from sklearn.svm import SVC
classifier= SVC(kernel='rbf',random_state=0)
classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)

In [31]:
cm=confusion_matrix(y_test,y_pred)
print(cm)
score = accuracy_score( y_test, y_pred)
print(score)

[[8893   41]
 [ 428  227]]
0.9510897903848159


Deep Learning Classification

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, SpatialDropout1D
from keras.layers import Bidirectional

In [33]:
max_features = 220
tokenizer = Tokenizer(num_words = max_features, split = (' '))
tokenizer.fit_on_texts(train['tweet'].values)
X = tokenizer.texts_to_sequences(train['tweet'].values)
# making all the tokens into same sizes using padding.
X = pad_sequences(X, maxlen = max_features)
X.shape

(31962, 220)

In [34]:
Y = train['label'].values


In [35]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length = X.shape[1], trainable=False))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [36]:
model.fit(X, Y,batch_size=1500,epochs = 5)

Epoch 1/5
22/22 [==============================] - 179s 8s/step - loss: 0.3700 - accuracy: 0.8942
Epoch 2/5
22/22 [==============================] - 163s 7s/step - loss: 0.2588 - accuracy: 0.9299
Epoch 3/5
22/22 [==============================] - 168s 8s/step - loss: 0.2573 - accuracy: 0.9299
Epoch 4/5
22/22 [==============================] - 163s 7s/step - loss: 0.2556 - accuracy: 0.9299
Epoch 5/5
22/22 [==============================] - 163s 7s/step - loss: 0.2539 - accuracy: 0.9299


In [37]:
prediction = model.predict_classes(X)

In [38]:
from sklearn.metrics import accuracy_score
score = accuracy_score(Y, prediction)
print(score)

0.9298542018647143


Using Vader Pre-trained model

In [39]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [40]:
train['score']=train['tweet'].apply(lambda tweet: sid.polarity_scores(tweet))
train.head()


,id,label,tweet,score
0,1,0,father dysfunct selfish drag kid dysfunct,"{'neg': 0.556, 'neu': 0.444, 'pos': 0.0, 'comp..."
1,2,0,thank lyft credit can't caus offer wheelchair ...,"{'neg': 0.0, 'neu': 0.611, 'pos': 0.389, 'comp..."
2,3,0,bihday majesti,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,4,0,model love take time,"{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'comp..."
4,5,0,factsguid societi motiv,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [41]:
train['compound']  = train['score'].apply(lambda score_dict: score_dict['compound'])
train.head()

,id,label,tweet,score,compound
0,1,0,father dysfunct selfish drag kid dysfunct,"{'neg': 0.556, 'neu': 0.444, 'pos': 0.0, 'comp...",-0.6124
1,2,0,thank lyft credit can't caus offer wheelchair ...,"{'neg': 0.0, 'neu': 0.611, 'pos': 0.389, 'comp...",0.6249
2,3,0,bihday majesti,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
3,4,0,model love take time,"{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'comp...",0.6369
4,5,0,factsguid societi motiv,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000


In [42]:
train['comp_score'] = train['compound'].apply(lambda c: 1 if c >0 else 0)

train.head()

,id,label,tweet,score,compound,comp_score
0,1,0,father dysfunct selfish drag kid dysfunct,"{'neg': 0.556, 'neu': 0.444, 'pos': 0.0, 'comp...",-0.6124,0
1,2,0,thank lyft credit can't caus offer wheelchair ...,"{'neg': 0.0, 'neu': 0.611, 'pos': 0.389, 'comp...",0.6249,1
2,3,0,bihday majesti,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0
3,4,0,model love take time,"{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'comp...",0.6369,1
4,5,0,factsguid societi motiv,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0


In [43]:
from sklearn.metrics import accuracy_score
score = accuracy_score(train['label'], train['comp_score'])
print(score)

0.553813904011013
